**get model coordinates**

In [1]:
import plotly.express as px
import pandas as pd
import numpy as np

coor = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Coron/met_ model/distan_lat42.58lon-8.8046p3R4Km.csv")
px.set_mapbox_access_token("pk.eyJ1IjoiZ3JhbmFudHVpbiIsImEiOiJja3B4dGU4OTkwMTFmMm9ycnNhMjJvaGJqIn0.VWzx_PkD9A5cSUVsn_ijCA")
px.scatter_mapbox(coor, hover_data=['distance'],lat='lat', lon='lon',color='distance', title="nearest points",
                           color_continuous_scale=px.colors.cyclical.IceFire,)

**get station and sample max gust hour before**

In [2]:
station = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Coron/station_data/station.csv", usecols = ["time","gust_o"],parse_dates=["time"]).set_index("time")
print("raw data\n",station.head(24))

#show duplicated
df_duplicate = station[station.index.duplicated()]
print("duplicated")
print(df_duplicate)

#delete duplicated and resample
station = station[~station.index.duplicated(keep='first')]
station = station.resample("H",label="right",closed="right").max()

print("Max gust hour before")
print(station.head(4))
print("Variable statistics")
print(station.describe())

raw data
                         gust_o
time                          
2018-01-01 00:10:00   6.666672
2018-01-01 00:20:00   6.138894
2018-01-01 00:30:00   7.472228
2018-01-01 00:40:00   6.833339
2018-01-01 00:50:00   7.333339
2018-01-01 01:00:00   8.527785
2018-01-01 01:10:00   9.138896
2018-01-01 01:20:00   8.000006
2018-01-01 01:30:00   8.777785
2018-01-01 01:40:00   9.222230
2018-01-01 01:50:00   8.361118
2018-01-01 02:00:00   8.583340
2018-01-01 02:10:00   9.583341
2018-01-01 02:20:00   9.222230
2018-01-01 02:30:00   8.638896
2018-01-01 02:40:00   9.694452
2018-01-01 02:50:00   8.638896
2018-01-01 03:00:00   8.472229
2018-01-01 03:10:00   8.055562
2018-01-01 03:20:00   8.250007
2018-01-01 03:30:00  11.416676
2018-01-01 03:40:00  12.583343
2018-01-01 03:50:00   9.055563
2018-01-01 04:00:00  10.916675
duplicated
                        gust_o
time                          
2018-03-25 03:00:00   2.527780
2018-03-25 03:10:00   6.166672
2018-03-25 03:20:00   6.222227
2018-03-25 03:30:0

**get meteorologic model d2**

In [3]:
#ria forecast 
met_model = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Coron/met_ model/lat42.58lon-8.8046p3R4KmD2.csv",parse_dates=["time"]).drop(columns=["Unnamed: 0"]).set_index("time")
met_model.drop_duplicates().info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 43824 entries, 2018-01-03 00:00:00 to 2023-01-03 23:00:00
Data columns (total 75 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   dir0         43824 non-null  float64
 1   snow_prec0   43824 non-null  float64
 2   snowlevel0   43824 non-null  float64
 3   mod0         43824 non-null  float64
 4   wind_gust0   43824 non-null  float64
 5   mslp0        43824 non-null  float64
 6   temp0        43824 non-null  float64
 7   rh0          43824 non-null  float64
 8   visibility0  43824 non-null  float64
 9   lhflx0       43824 non-null  float64
 10  lwflx0       43824 non-null  float64
 11  conv_prec0   43824 non-null  float64
 12  prec0        43824 non-null  float64
 13  swflx0       43824 non-null  float64
 14  shflx0       43824 non-null  float64
 15  cape0        43824 non-null  float64
 16  cin0         43824 non-null  float64
 17  cfh0         43824 non-null  float64
 18  cfl0       

**Concatenate and dropnan**

In [4]:
df_all = pd.concat([station,met_model],axis=1).dropna()
df_all.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 43108 entries, 2018-01-03 00:00:00 to 2022-12-31 00:00:00
Data columns (total 76 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   gust_o       43108 non-null  float64
 1   dir0         43108 non-null  float64
 2   snow_prec0   43108 non-null  float64
 3   snowlevel0   43108 non-null  float64
 4   mod0         43108 non-null  float64
 5   wind_gust0   43108 non-null  float64
 6   mslp0        43108 non-null  float64
 7   temp0        43108 non-null  float64
 8   rh0          43108 non-null  float64
 9   visibility0  43108 non-null  float64
 10  lhflx0       43108 non-null  float64
 11  lwflx0       43108 non-null  float64
 12  conv_prec0   43108 non-null  float64
 13  prec0        43108 non-null  float64
 14  swflx0       43108 non-null  float64
 15  shflx0       43108 non-null  float64
 16  cape0        43108 non-null  float64
 17  cin0         43108 non-null  float64
 18  cfh0       

**best correlation**

In [5]:
df_all.corr()["gust_o"].sort_values(ascending=False).head(10)

gust_o        1.000000
mod0          0.706405
wind_gust1    0.698077
mod1          0.685959
wind_gust0    0.685502
wind_gust2    0.682954
mod2          0.671201
lhflx2        0.278035
lhflx1        0.269164
lhflx0        0.240386
Name: gust_o, dtype: float64

**Mean absolute error in m/s**

In [6]:
from sklearn.metrics import mean_absolute_error
import plotly.express as px

print("mean absolute error mod0:", round(mean_absolute_error(df_all.gust_o, df_all.mod0),2))
print("mean absolute error wg1:", round(mean_absolute_error(df_all.gust_o, df_all.wind_gust1),2))
print("mean absolute error wg0:", round(mean_absolute_error(df_all.gust_o, df_all.wind_gust0),2))
print("mean absolute error wg2:", round(mean_absolute_error(df_all.gust_o, df_all.wind_gust2),2))
print("mean absolute error mod1:", round(mean_absolute_error(df_all.gust_o, df_all.mod1),2))

diff =  df_all["wind_gust1"]-df_all["gust_o"]
print("\nDifferences wind gust1 and gust observed")
print(diff.describe())
print("\nDifferences")
fig = px.line(diff)
fig.show()


mean absolute error mod0: 3.69
mean absolute error wg1: 2.67
mean absolute error wg0: 2.74
mean absolute error wg2: 2.75
mean absolute error mod1: 3.21

Differences wind gust1 and gust observed
count    43108.000000
mean        -0.150470
std          3.490420
min        -19.907580
25%         -2.340567
50%         -0.351630
75%          1.832755
max         19.415055
dtype: float64

Differences


**LGBMRegressor**

In [7]:
from lightgbm.sklearn import LGBMRegressor
from sklearn.model_selection import train_test_split

#numpy randomseed
np.random.seed(1)

# target variable
Y = df_all.gust_o
X = df_all[["wind_gust0","mod0","wind_gust1","mod1","wind_gust2","mod2","prec0"]]

MAE = []
for t in range (0,20):
  X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size=0.1, )
  ml_model = LGBMRegressor(n_estimators=200).fit(X_train,y_train)
  y_pred = ml_model.predict(X_test)
  MAE.append(round(mean_absolute_error(y_test,y_pred),2))
  print("machine learning number:",t)
  df = pd.DataFrame({"y_test":y_test},index=y_test.index)
  df["y_pred"] = y_pred
  df["diff"] = df["y_pred"]-df["y_test"]
  print(df.describe())
print("\nDifferences last machine learning model")
fig = px.line(df["diff"].sort_index())
fig.show()  
print("Mean absolute value all machine learning models")
df_r=pd.DataFrame({"MAE":MAE,});
df_r.describe()


machine learning number: 0
            y_test       y_pred         diff
count  4311.000000  4311.000000  4311.000000
mean      7.578275     7.643592     0.065317
std       4.396801     3.409968     2.899179
min       0.000000     1.051624   -15.161804
25%       3.916670     4.705198    -1.703754
50%       7.111117     7.131259     0.197938
75%      10.583342     9.963095     1.896234
max      25.805576    19.917005    10.794787
machine learning number: 1
            y_test       y_pred         diff
count  4311.000000  4311.000000  4311.000000
mean      7.655906     7.584711    -0.071195
std       4.316621     3.281485     2.910472
min       0.000000     1.693385   -15.463883
25%       4.083337     4.784186    -1.814724
50%       7.166672     7.076929     0.094342
75%      10.638897     9.787497     1.761766
max      23.833352    20.507147    13.962961
machine learning number: 2
            y_test       y_pred         diff
count  4311.000000  4311.000000  4311.000000
mean      7.585691 

Mean absolute value all machine learning models


,MAE
count,20.000000
mean,2.219000
std,0.028636
min,2.150000
25%,2.200000
50%,2.215000
75%,2.240000
max,2.270000


**Shapiro test normality MAE samples tested**

In [8]:
from scipy.stats import shapiro

sta, p = shapiro(MAE)
print("\np value:",p)
print("significance level alpha=0.01")

alpha = 0.01
if p>alpha:
  print("Accept null hypothesis: normal distribution variable MAE in the samples")
else:
  print("Reject null Hypothesis. Alternative hypothesis variable MAE no normal distribution in samples")  


p value: 0.6513538956642151
significance level alpha=0.01
Accept null hypothesis: normal distribution variable MAE in the samples


**T test (1sample) compare machine learning mean absolute error in a sample (n=20) versus meteorological model mean absolute error at point 1  (wind_gust1 = 2.67). The alternative hypothesis is: the sample mean absolute error of  the sample is less than the given population mean absolute error (popmean). Significance level (alpha) = 0.01**

In [9]:
from scipy import stats

alpha = 0.01
sta0, p_value0 = stats.ttest_1samp(MAE, popmean=2.67, alternative="less")

print("MAE in test sample:",MAE)
print("p value:",p_value0)

if p_value0 < alpha:    
    print("\nwe are rejecting null hypothesis  H0: Mean Absolute Error machine learning = Mean Absolute Error meteorological model wind_gust1")
    print("\nwe are accepting alternative hypothesis: Mean Absolute Error machine learning is less than Mean Absolute Error meteorological model wind_gust1")
else:
    print("\nwe are accepting null hypothesis:  H0:  Mean Absolute Error machine learning = Mean Absolute Error meteorological model wind_gust1")

MAE in test sample: [2.25, 2.24, 2.25, 2.23, 2.2, 2.2, 2.21, 2.24, 2.2, 2.21, 2.25, 2.21, 2.21, 2.23, 2.27, 2.15, 2.24, 2.19, 2.22, 2.18]
p value: 9.573105220837431e-25

we are rejecting null hypothesis  H0: Mean Absolute Error machine learning = Mean Absolute Error meteorological model wind_gust1

we are accepting alternative hypothesis: Mean Absolute Error machine learning is less than Mean Absolute Error meteorological model wind_gust1


**Add time variables**

In [10]:

df_all["hour"] = df_all.index.hour
df_all["month"] = df_all.index.month
df_all["dayofyear"] = df_all.index.dayofyear
df_all["weekofyear"] = df_all.index.isocalendar().week.astype(int)

#numpy randomseed
np.random.seed(1)

# target variable
Y = df_all.gust_o
X = df_all[["wind_gust0","mod0","wind_gust1","mod1","wind_gust2","mod2","hour",
            "month","dayofyear","weekofyear"]]

MAE_t = []
for t in range (0,20):
  X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size=0.1, )
  ml_model = LGBMRegressor(n_estimators=200).fit(X_train,y_train)
  y_pred = ml_model.predict(X_test)
  MAE_t.append(round(mean_absolute_error(y_test,y_pred),2))
  print("machine learning number:",t)
  df = pd.DataFrame({"y_test":y_test},index=y_test.index)
  df["y_pred"] = y_pred
  df["diff"] = df["y_pred"]-df["y_test"]
  print(df.describe())
print("\nDifferences last machine learning model")
fig = px.line(df["diff"].sort_index())
fig.show()  
print("Mean absolute value all machine learning models")
df_r=pd.DataFrame({"MAE_t":MAE_t,});
df_r.describe()



machine learning number: 0
            y_test       y_pred         diff
count  4311.000000  4311.000000  4311.000000
mean      7.578275     7.656551     0.078277
std       4.396801     3.456396     2.691064
min       0.000000     1.513005   -15.063046
25%       3.916670     4.832742    -1.539966
50%       7.111117     7.128302     0.253523
75%      10.583342    10.043255     1.784928
max      25.805576    21.155411     9.824141
machine learning number: 1
            y_test       y_pred         diff
count  4311.000000  4311.000000  4311.000000
mean      7.655906     7.586615    -0.069291
std       4.316621     3.347103     2.712789
min       0.000000     1.841935   -14.490451
25%       4.083337     4.813315    -1.732238
50%       7.166672     7.029588     0.129647
75%      10.638897     9.877254     1.639515
max      23.833352    20.364981    13.175142
machine learning number: 2
            y_test       y_pred         diff
count  4311.000000  4311.000000  4311.000000
mean      7.585691 

Mean absolute value all machine learning models


,MAE_t
count,20.000000
mean,2.069500
std,0.022821
min,2.030000
25%,2.050000
50%,2.075000
75%,2.080000
max,2.120000


**T test two related samples. This is a test for the null hypothesis that two related samples have identical mean absolute error. Alternative hypothesis the  mean absolute error from the sample with time variables is less than the mean absolute error of from the sample with time variables. Significance level (alpha) = 0.01**

In [11]:
from scipy import stats

alpha = 0.01
sta, p_value = stats.ttest_rel(MAE, MAE_t, alternative="greater")

print("\nMAE without time variables:", np.round(MAE,3))
print("\nMAE with time variables:", np.round(MAE_t,3))
print("\nalpha: ",alpha)
print("\np_value point: ",p_value)

if p_value < alpha:    
    print("\nwe are rejecting null hypothesis  H0: MAE machine learning without time variables = MAE machine learning with time variables")
    print("\nwe are accepting alternative hypothesis: the MAE of the sample with time variables inputs is less than MAE of the sample without time variables inputs")
else:
    print("\nwe are accepting null hypothesis:  H0: MAE machine learning without time variables = MAE machine learning with time variables")


MAE without time variables: [2.25 2.24 2.25 2.23 2.2  2.2  2.21 2.24 2.2  2.21 2.25 2.21 2.21 2.23
 2.27 2.15 2.24 2.19 2.22 2.18]

MAE with time variables: [2.09 2.07 2.08 2.08 2.06 2.06 2.05 2.08 2.08 2.08 2.1  2.05 2.06 2.05
 2.12 2.03 2.09 2.05 2.08 2.03]

alpha:  0.01

p_value point:  1.3101942361292037e-20

we are rejecting null hypothesis  H0: MAE machine learning without time variables = MAE machine learning with time variables

we are accepting alternative hypothesis: the MAE of the sample with time variables inputs is less than MAE of the sample without time variables inputs


**Save algorithm**

In [12]:
import pickle
from sklearn.pipeline import Pipeline


estimators = [('ml_model', ml_model)]
pipe = Pipeline(estimators)
print(pipe)
al_file={"x_var":X.columns,"coor":coor,"pipe":pipe}
pickle.dump(al_file, open("/content/drive/MyDrive/Colab Notebooks/Coron/algorithms/gust_coron_d2.al", 'wb'))

Pipeline(steps=[('ml_model', LGBMRegressor(n_estimators=200))])
